In [3]:
import json
import os

from tqdm import tqdm

from desktop_env.envs.desktop_env import DesktopEnv

In [4]:
test_all_meta_path = "D:\jcy\OSWorld-main/evaluation_examples/test_all.json"

with open(test_all_meta_path, "r", encoding="utf-8") as f:
    test_all_meta = json.load(f)
left_info = ""
for domain in test_all_meta:
    left_info += f"{domain}: {len(test_all_meta[domain])}\n"
print(f"Left tasks:\n{left_info}")

env = DesktopEnv(
    path_to_vm=r"D:\jcy\OSWorld-main/Ubuntu/Ubuntu.vmx",
    snapshot_name='os-copilot_v0',
    action_space="pyautogui",
    require_a11y_tree=True)

Left tasks:
chrome: 46
gimp: 26
libreoffice_calc: 47
libreoffice_impress: 47
libreoffice_writer: 23
multi_apps: 101
os: 24
thunderbird: 15
vlc: 17
vs_code: 23



In [7]:
i = 0
domain = 'vs_code'
task = test_all_meta[domain]
example_id = task[i]
example_id = '847a96b6-df94-4927-97e6-8cc9ea66ced7'
config_file = os.path.join("D:\jcy\OSWorld-main/evaluation_examples", f"examples/{domain}/{example_id}.json")
with open(config_file, "r", encoding="utf-8") as f:
    example = json.load(f)
print(example['id'], example['instruction'])



obs = env.reset(task_config=example)
obs, reward, done, info = env.step("pyautogui.rightClick()")

847a96b6-df94-4927-97e6-8cc9ea66ced7 Please help me open two workspaces "/home/user/workspace1.code-workspace" and "/home/user/workspace2.code-workspace" simultaneously in the same window.


In [ ]:
def run_single_example(agent, env, example, max_steps, instruction, args, example_result_dir, scores):
    agent.reset()
    obs = env.reset(task_config=example)
    done = False
    step_idx = 0
    env.controller.start_recording()
    # str_table = wandb.Table(columns=["Screenshot", "A11T", "Modle Response", "Action", "Action timestamp", "Done"])
    while not done and step_idx < max_steps:
        response, actions = agent.predict(
            instruction,
            obs
        )
        for action in actions:
            # Capture the timestamp before executing the action
            action_timestamp = datetime.datetime.now().strftime("%Y%m%d@%H%M%S")
            logger.info("Step %d: %s", step_idx + 1, action)
            obs, reward, done, info = env.step(action, args.sleep_after_execution)

            logger.info("Reward: %.2f", reward)
            logger.info("Done: %s", done)
            # Save screenshot and trajectory information
            with open(os.path.join(example_result_dir, f"step_{step_idx + 1}_{action_timestamp}.png"),
                      "wb") as _f:
                with open(obs['screenshot'], "rb") as __f:
                    screenshot = __f.read()
                _f.write(screenshot)
            # get a11tree and save to wandb
            # thisrun_a11tree = env.controller.get_accessibility_tree()
            # str_table.add_data(wandb.Image(data_or_path=os.path.join(example_result_dir, f"step_{step_idx + 1}_{action_timestamp}.png"), caption=f"step_{step_idx + 1}_{action_timestamp}"),
            #                 thisrun_a11tree,
            #                 response, action, action_timestamp, done)
            # run.log({"Reward": reward})
            with open(os.path.join(example_result_dir, "traj.jsonl"), "a") as f:
                f.write(json.dumps({
                    "step_num": step_idx + 1,
                    "action_timestamp": action_timestamp,
                    "action": action,
                    "reward": reward,
                    "done": done,
                    "info": info,
                    "screenshot_file": f"step_{step_idx + 1}_{action_timestamp}.png"
                }))
                f.write("\n")
            if done:
                logger.info("The episode is done.")
                break
        step_idx += 1
    # run.log({"str_trajectory": str_table})
    result = env.evaluate()
    logger.info("Result: %.2f", result)
    scores.append(result)
    with open(os.path.join(example_result_dir, "result.txt"), "w", encoding="utf-8") as f:
        f.write(f"{result}\n")
    env.controller.end_recording(os.path.join(example_result_dir, "recording.mp4"))
    # run.log({"Result": result})